<a href="https://colab.research.google.com/github/khromets/project_f/blob/main/nfl_scores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import pandas as pd
import requests
import datetime
import pytz
from google.colab import drive

In [ ]:
url='http://site.api.espn.com/apis/site/v2/sports/football/nfl/scoreboard'
response = requests.get(url)
data = json.loads(response.text)

In [ ]:
year=[]
week=[]
game_short_name=[]
score=[]
game_date=[]
home_full_name=[]
home_team_record=[]
away_full_name=[]
away_team_record=[]
game_status=[]
uid=[]
insert_date=[]


for x in data['events']:
  # x['name'] - match name
  item_1 = x['competitions'][0]
  item_2=item_1['competitors'][0] # there are two teams (competitors). 0 - home team, 1 - away team
  item_3=item_1['competitors'][1] # there are two teams (competitors). 0 - home team, 1 - away team
  
  year.append(data['season']['year'])
  week.append(data['week']['number'])
  game_short_name.append(x['shortName'])
  score.append(item_2['score']+'-'+item_3['score'])
  game_date.append(x['date'])
  home_full_name.append(item_2['team']['displayName'])
  home_team_record.append(item_2['records'][0]['summary'])
  away_full_name.append(item_3['team']['displayName'])
  away_team_record.append(item_3['records'][0]['summary'])
  game_status.append(item_1['status']['type']['name'])
  uid.append(x['uid'])
  insert_date.append(datetime.datetime.now(pytz.timezone("America/New_York")))

In [ ]:
data = {'year': year,
        'week': week,
        'game_short_name': game_short_name,
        'score': score,
        'game_date': game_date,
        'home_full_name': home_full_name,
        'home_team_record': home_team_record,
        'away_full_name': away_full_name,
        'away_team_record': away_team_record,
        'game_status': game_status,
        'uid': uid,
        'insert_date': insert_date
        }
df = pd.DataFrame(data)
print(df)

   year  week game_short_name score          game_date       home_full_name  \
0  2022     3        SF @ PHI   0-0  2023-01-29T20:00Z  Philadelphia Eagles   
1  2022     3        CIN @ KC   0-0  2023-01-29T23:30Z   Kansas City Chiefs   

  home_team_record       away_full_name away_team_record       game_status  \
0             14-3  San Francisco 49ers             13-4  STATUS_SCHEDULED   
1             14-3   Cincinnati Bengals             12-4  STATUS_SCHEDULED   

                     uid                      insert_date  
0  s:20~l:28~e:401438009 2023-01-28 15:17:23.195092-05:00  
1  s:20~l:28~e:401438008 2023-01-28 15:17:23.195158-05:00  


In [ ]:
#drive.mount('drive')

In [ ]:
#drive.mount('/content/drive', force_remount=True)

In [ ]:
df.to_csv('/content/drive/My Drive/NFL_scores/nfl_score_'+datetime.datetime.now(pytz.timezone("America/New_York")).strftime("%Y%m%d_%H%M%S")+'.csv',index=False)

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account

In [ ]:
credentials = service_account.Credentials.from_service_account_file('/content/drive/My Drive/NFL_scores/project-f-374403-cf1535d2706e.json')
project_id = 'project-f-374403'
client = bigquery.Client(credentials= credentials,project=project_id)

In [ ]:
sql ="""
   SELECT *
   FROM `project-f-374403.evgenii.nfl_scores`
   LIMIT 1000 """

In [ ]:
pct_overlap_terms_by_days_apart = client.query(sql).to_dataframe()

pct_overlap_terms_by_days_apart.head()

,year,week,game_short_name,score,game_date,home_full_name,home_team_record,away_full_name,away_team_record,game_status,uid,insert_date
0,2022,1,DAL @ TB,0-0,2023-01-17T01:15Z,Tampa Bay Buccaneers,8-9,Dallas Cowboys,12-5,STATUS_SCHEDULED,s:20~l:28~e:401438003,2023-01-13 03:20:51.576514+00:00
1,2022,1,DAL @ TB,14-31,2023-01-17T01:15Z,Tampa Bay Buccaneers,8-9,Dallas Cowboys,12-5,STATUS_FINAL,s:20~l:28~e:401438003,2023-01-18 02:28:31.089985+00:00
2,2022,1,LAC @ JAX,0-0,2023-01-15T01:15Z,Jacksonville Jaguars,9-8,Los Angeles Chargers,10-7,STATUS_SCHEDULED,s:20~l:28~e:401437998,2023-01-13 03:20:51.576402+00:00
3,2022,1,LAC @ JAX,31-30,2023-01-15T01:15Z,Jacksonville Jaguars,9-8,Los Angeles Chargers,10-7,STATUS_FINAL,s:20~l:28~e:401437998,2023-01-18 02:28:31.089878+00:00
4,2022,1,BAL @ CIN,0-0,2023-01-16T01:15Z,Cincinnati Bengals,12-4,Baltimore Ravens,10-7,STATUS_SCHEDULED,s:20~l:28~e:401437999,2023-01-13 03:20:51.576488+00:00


In [ ]:
from pandas_gbq import to_gbq

In [ ]:
# convert the dataframe to a BigQuery table
to_gbq(df, 'project-f-374403.evgenii.nfl_scores', project_id=project_id, if_exists='append', credentials=credentials)

100%|██████████| 1/1 [00:00<00:00, 1025.00it/s]
